In [ ]:
!pip install -q gradio runorm transformers torch torchaudio
!pip install -q huggingface_hub soundfile pydub silero-stress
!pip install -q git+https://github.com/mikhail2013ru/F5-TTS-RUS.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# НАЖАТЬ RESTART SESSION

In [ ]:
#!/usr/bin/env python3
import time
import re
import os
import gc
import tempfile
import traceback
import json
import threading
import numpy as np
import math
from pydub import AudioSegment
from pydub.effects import normalize
from pathlib import Path

import gradio as gr
import soundfile as sf
import torch
import torchaudio

from huggingface_hub import login, hf_hub_download, snapshot_download

# ================ УЛУЧШЕННАЯ АВТОРИЗАЦИЯ HUGGINGFACE ================
def setup_huggingface_auth():
    """Настройка авторизации HuggingFace для доступа к закрытым репозиториям"""
    print("\n" + "="*60)
    print("🔐 НАСТРОЙКА АВТОРИЗАЦИИ HUGGINGFACE")
    print("="*60)
    
    hf_token = None
    token_source = "Не найден"
    
    if os.environ.get("HF_TOKEN"):
        hf_token = os.environ["HF_TOKEN"]
        token_source = "переменная окружения HF_TOKEN"
    elif os.path.exists("/root/.cache/huggingface/token"):
        try:
            with open("/root/.cache/huggingface/token", "r") as f:
                hf_token = f.read().strip()
            if hf_token:
                token_source = "файл /root/.cache/huggingface/token"
        except Exception as e:
            print(f"⚠️ Ошибка чтения файла токена: {e}")
    else:
        try:
            from google.colab import userdata
            hf_token = userdata.get('HF_TOKEN')
            if hf_token:
                token_source = "секреты Colab"
        except ImportError:
            print("⚠️ Не в Colab или google.colab недоступен")
        except Exception as e:
            print(f"⚠️ Ошибка получения токена из Colab: {e}")
    
    if not hf_token:
        hf_cache_token = os.path.expanduser("~/.cache/huggingface/token")
        if os.path.exists(hf_cache_token):
            try:
                with open(hf_cache_token, "r") as f:
                    hf_token = f.read().strip()
                if hf_token:
                    token_source = f"файл {hf_cache_token}"
            except:
                pass
    
    if hf_token:
        try:
            print(f"✅ Токен найден в {token_source}")
            
            if not hf_token.startswith("hf_"):
                print(f"⚠️ Предупреждение: Токен может быть неверного формата. Должен начинаться с 'hf_'")
            
            login(token=hf_token)
            os.environ["HF_TOKEN"] = hf_token
            
            try:
                os.makedirs("/root/.cache/huggingface", exist_ok=True)
                with open("/root/.cache/huggingface/token", "w") as f:
                    f.write(hf_token)
                print("✅ Токен сохранен для будущих запусков")
            except Exception as e:
                print(f"⚠️ Не удалось сохранить токен в файл: {e}")
            
            print("✅ Авторизация HuggingFace успешна!")
            return hf_token
            
        except Exception as e:
            error_msg = str(e)
            print(f"❌ Ошибка авторизации: {error_msg}")
            
            if "401" in error_msg:
                print("❌ Неверный токен авторизации. Проверьте правильность токена.")
            elif "400" in error_msg:
                print("❌ Неверный формат токена. Токен должен начинаться с 'hf_'.")
            elif "connection" in error_msg.lower():
                print("❌ Проблемы с подключением к интернету. Проверьте соединение.")
            
            return None
    else:
        print("ℹ️  Токен не найден. Продолжаем без авторизации.")
        print("   Закрытые репозитории будут недоступны.")
        
        print("\n" + "-"*60)
        print("📋 Как добавить токен:")
        print("1. Получите токен на https://huggingface.co/settings/tokens")
        print("2. Добавьте токен одним из способов:")
        print("   • В Colab: добавьте в секреты с ключом 'HF_TOKEN'")
        print("   • В переменные окружения: export HF_TOKEN='ваш_токен'")
        print("   • В файл: echo 'ваш_токен' > /root/.cache/huggingface/token")
        print("-"*60)
        
        return None

# Вызываем авторизацию в начале
print("\n" + "="*60)
print("🚀 ЗАПУСК ESpeech-TTS")
print("="*60)

HF_TOKEN = setup_huggingface_auth()
print("="*60 + "\n")

# ================ ГЛОБАЛЬНЫЕ ФЛАГИ ДЛЯ УПРАВЛЕНИЯ ГЕНЕРАЦИЕЙ ================
stop_generation = False
generation_thread = None
generation_event = threading.Event()
stop_requested = False

from silero_stress import load_accentor
from runorm import RUNorm
from f5_tts.infer.utils_infer import (
    infer_process,
    load_model,
    load_vocoder,
    preprocess_ref_audio_text,
    remove_silence_for_generated_wav,
    save_spectrogram,
    tempfile_kwargs,
)
from f5_tts.model import DiT

from transformers import WhisperProcessor, WhisperForConditionalGeneration

MODELS_CONFIG = {
    "ESpeech-TTS-1_SFT-95K": {
        "repo": "ESpeech/ESpeech-TTS-1_SFT-95K",
        "model_file": "espeech_tts_95k.pt",
        "vocab_file": "vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "ESpeech-TTS-1_RL-V1": {
        "repo": "ESpeech/ESpeech-TTS-1_RL-V1",
        "model_file": "espeech_tts_rlv1.pt",
        "vocab_file": "vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "ESpeech-TTS-1_RL-V2": {
        "repo": "ESpeech/ESpeech-TTS-1_RL-V2",
        "model_file": "espeech_tts_rlv2.pt",
        "vocab_file": "vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "F5TTS_v1_Base_v2_Misha24-10": {
        "repo": "Misha24-10/F5-TTS_RUSSIAN",
        "model_file": "F5TTS_v1_Base_v2/model_last_inference.safetensors",
        "vocab_file": "F5TTS_v1_Base/vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "F5TTS_v1_Base_v4_winter": {
        "repo": "Misha24-10/F5-TTS_RUSSIAN",
        "model_file": "F5TTS_v1_Base_v4_winter/model_212000.safetensors",
        "vocab_file": "F5TTS_v1_Base/vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "F5TTS_v1_Base_1.25M_SWivid": {
        "repo": "SWivid/F5-TTS",
        "model_file": "F5TTS_v1_Base/model_1250000.safetensors",
        "vocab_file": "F5TTS_v1_Base/vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
}

# Глобальные переменные для моделей
loaded_models = {}
loaded_vocoder = None
current_model_name = None
remember_seed = False
last_seed = -1

# Для хранения нормализованных записей
normalized_audio_cache = {}

# Словарь для локальных моделей
LOCAL_MODELS_FILE = "local_models.json"
local_models_config = {}

def load_local_models():
    """Загрузка локальных моделей из файла"""
    global local_models_config
    try:
        if os.path.exists(LOCAL_MODELS_FILE):
            with open(LOCAL_MODELS_FILE, 'r', encoding='utf-8') as f:
                local_models_config = json.load(f)
            print(f"✅ Загружено {len(local_models_config)} локальных моделей")
        else:
            local_models_config = {}
            print("✅ Файл локальных моделей не найден, создан новый")
    except Exception as e:
        print(f"❌ Ошибка загрузки локальных моделей: {e}")
        local_models_config = {}

def save_local_models():
    """Сохранение локальных моделей в файл"""
    try:
        with open(LOCAL_MODELS_FILE, 'w', encoding='utf-8') as f:
            json.dump(local_models_config, f, ensure_ascii=False, indent=2)
        print(f"✅ Сохранено {len(local_models_config)} локальных моделей")
    except Exception as e:
        print(f"❌ Ошибка сохранения локальных моделей: {e}")

load_local_models()

def get_all_models_config():
    """Получить объединенный конфиг моделей"""
    return {**MODELS_CONFIG, **local_models_config}

def clear_loaded_models():
    """Очистка загруженных моделей из памяти"""
    global loaded_models, loaded_vocoder, current_model_name, normalized_audio_cache
    
    removed_local_models = list(local_models_config.keys())
    
    loaded_models.clear()
    loaded_vocoder = None
    current_model_name = None
    normalized_audio_cache.clear()
    
    local_models_config.clear()
    save_local_models()
    
    torch.cuda.empty_cache()
    gc.collect()
    
    updated_models_config = get_all_models_config()
    new_model = list(updated_models_config.keys())[0] if updated_models_config else None
    
    removed_count = len(removed_local_models)
    status_message = f"✅ Удалено {removed_count} локальных моделей и выгружено из памяти"
    
    print(status_message)
    
    return (
        gr.update(value=f"<div class='model-status model-loaded'>{status_message}</div>", visible=True),
        gr.update(value=""),
        gr.update(value=""),
        gr.update(value=""),
        gr.update(choices=list(updated_models_config.keys()), value=new_model)
    )

def remove_local_model(model_name):
    """Удаление локальной модели из конфигурации"""
    global local_models_config
    
    if model_name in local_models_config:
        del local_models_config[model_name]
        save_local_models()
        
        updated_models_config = get_all_models_config()
        new_model = list(updated_models_config.keys())[0] if updated_models_config else None
        
        return (
            gr.update(choices=list(updated_models_config.keys()), value=new_model),
            gr.update(value=f"✅ Модель '{model_name}' удалена", visible=True)
        )
    else:
        return gr.update(), gr.update(value=f"❌ Модель '{model_name}' не найдена", visible=True)

def check_cuda_availability():
    """Check if CUDA is available and raise error if not"""
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available! This application requires GPU with CUDA support.")
    print(f"CUDA is available. Using device: {torch.cuda.get_device_name()}")

def parse_hf_path(hf_path):
    """Парсит HF путь в repo_id и filename"""
    if hf_path.startswith('hf://'):
        path_parts = hf_path.replace('hf://', '').split('/')
        if len(path_parts) >= 2:
            repo_id = '/'.join(path_parts[:2])
            filename = '/'.join(path_parts[2:])
            return repo_id, filename
    return None, None

def add_local_model(model_path, vocab_path, model_name):
    """Добавить локальную модель в конфигурацию"""
    global local_models_config, MODELS_CONFIG
    
    if not model_path or not vocab_path or not model_name:
        return gr.update(), gr.update(value="❌ Заполните все поля", visible=True)
    
    def validate_hf_path(file_path, file_type):
        if file_path.startswith('hf://'):
            repo_id, filename = parse_hf_path(file_path)
            if not repo_id or not filename:
                return False, f"❌ Неверный формат HF пути для {file_type}"
            
            if '/' not in repo_id or len(repo_id.split('/')) != 2:
                return False, f"❌ Неверный формат repo_id: {repo_id}"
                
            return True, None
        else:
            if os.path.exists(file_path):
                return True, None
            else:
                return False, f"❌ Файл {file_type} не найден: {file_path}"
    
    is_model_valid, model_error = validate_hf_path(model_path, "модели")
    if not is_model_valid:
        return gr.update(), gr.update(value=model_error, visible=True)
    
    is_vocab_valid, vocab_error = validate_hf_path(vocab_path, "словаря")
    if not is_vocab_valid:
        return gr.update(), gr.update(value=vocab_error, visible=True)
    
    local_models_config[model_name] = {
        "repo": "local",
        "model_file": model_path,
        "vocab_file": vocab_path,
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    }

    save_local_models()
    
    updated_models_config = {**MODELS_CONFIG, **local_models_config}
    
    return gr.update(
        choices=list(updated_models_config.keys()),
        value=model_name
    ), gr.update(value=f"✅ Локальная модель '{model_name}' добавена", visible=True)

def check_token_validity(token):
    """Проверить валидность токена"""
    if not token:
        return False, "Токен пустой"
    
    if not token.startswith("hf_"):
        return False, "Токен должен начинаться с 'hf_'"
    
    if len(token) < 20:
        return False, "Токен слишком короткий"
    
    return True, "Формат токена корректный"

def update_hf_token(new_token):
    """Обновление токена HuggingFace"""
    global HF_TOKEN
    
    if not new_token:
        return gr.update(value="❌ Введите токен", visible=True)
    
    is_valid, validation_msg = check_token_validity(new_token)
    if not is_valid:
        return gr.update(value=f"❌ {validation_msg}", visible=True)
    
    try:
        print(f"🔄 Попытка авторизации с новым токеном...")
        
        login(token=new_token)
        HF_TOKEN = new_token
        
        try:
            os.makedirs("/root/.cache/huggingface", exist_ok=True)
            with open("/root/.cache/huggingface/token", "w") as f:
                f.write(new_token)
            print(f"✅ Токен сохранен в файл")
        except Exception as file_error:
            print(f"⚠️ Не удалось сохранить токен в файл: {file_error}")
        
        os.environ["HF_TOKEN"] = new_token
        
        print(f"✅ Токен обновлен")
        return gr.update(value="✅ Токен успешно обновлен и сохранен!", visible=True)
    
    except Exception as e:
        error_msg = str(e)
        print(f"❌ Ошибка обновления токена: {error_msg}")
        
        if "401" in error_msg:
            detailed_msg = "❌ Неверный токен авторизации. Проверьте правильность токена."
        elif "400" in error_msg:
            detailed_msg = "❌ Неверный формат токена."
        elif "connection" in error_msg.lower():
            detailed_msg = "❌ Проблемы с подключением к интернету. Проверьте соединение."
        else:
            detailed_msg = f"❌ Ошибка: {str(e)[:100]}"
        
        return gr.update(value=detailed_msg, visible=True)

def add_local_model_with_auth(model_path, vocab_path, model_name, hf_token=None):
    """Добавить локальную модель с поддержкой авторизации"""
    global local_models_config, MODELS_CONFIG, HF_TOKEN
    
    if not model_path or not vocab_path or not model_name:
        return gr.update(), gr.update(value="❌ Заполните все поля", visible=True)
    
    use_token = hf_token if hf_token else HF_TOKEN
    
    def validate_hf_path(file_path, file_type):
        if file_path.startswith('hf://'):
            repo_id, filename = parse_hf_path(file_path)
            if not repo_id or not filename:
                return False, f"❌ Неверный формат HF пути для {file_type}"
            
            if '/' not in repo_id or len(repo_id.split('/')) != 2:
                return False, f"❌ Неверный формат repo_id: {repo_id}"
                
            return True, None
        else:
            if os.path.exists(file_path):
                return True, None
            else:
                return False, f"❌ Файл {file_type} не найден: {file_path}"
    
    is_model_valid, model_error = validate_hf_path(model_path, "модели")
    if not is_model_valid:
        return gr.update(), gr.update(value=model_error, visible=True)
    
    is_vocab_valid, vocab_error = validate_hf_path(vocab_path, "словаря")
    if not is_vocab_valid:
        return gr.update(), gr.update(value=vocab_error, visible=True)
    
    def download_hf_file_with_token(hf_path, file_type):
        if hf_path.startswith('hf://'):
            try:
                repo_id, filename = parse_hf_path(hf_path)
                if not repo_id or not filename:
                    raise ValueError(f"Invalid HF path: {hf_path}")
                
                print(f"Скачивание {file_type} из {repo_id}/{filename}")
                
                downloaded_path = hf_hub_download(
                    repo_id=repo_id,
                    filename=filename,
                    repo_type="model",
                    token=use_token
                )
                print(f"✅ {file_type} скачан: {downloaded_path}")
                return downloaded_path
                
            except Exception as e:
                error_msg = str(e)
                if "401" in error_msg or "403" in error_msg:
                    if use_token is None:
                        raise PermissionError(f"❌ Репозиторий {repo_id} приватный. Требуется токен авторизации!")
                    else:
                        raise PermissionError(f"❌ Нет доступа к {repo_id}. Проверьте токен или права доступа.")
                elif "404" in error_msg:
                    raise FileNotFoundError(f"❌ Файл {filename} не найден в репозитории {repo_id}")
                else:
                    raise e
        else:
            if not os.path.exists(hf_path):
                raise FileNotFoundError(f"Файл не найден: {hf_path}")
            return hf_path
    
    try:
        downloaded_model = download_hf_file_with_token(model_path, "модели")
        downloaded_vocab = download_hf_file_with_token(vocab_path, "словаря")
        
        local_models_config[model_name] = {
            "repo": "local",
            "model_file": downloaded_model,
            "vocab_file": downloaded_vocab,
            "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
        }

        save_local_models()
        
        updated_models_config = {**MODELS_CONFIG, **local_models_config}
        
        return gr.update(
            choices=list(updated_models_config.keys()),
            value=model_name
        ), gr.update(value=f"✅ Локальная модель '{model_name}' добавлена", visible=True)
        
    except Exception as e:
        error_msg = f"❌ Ошибка: {str(e)}"
        print(f"Ошибка добавления модели: {traceback.format_exc()}")
        return gr.update(), gr.update(value=error_msg, visible=True)

def load_model_with_progress(model_name, progress_callback=None):
    """Загрузка модели с индикатором прогресса"""
    global loaded_models, current_model_name, stop_generation
    
    if stop_generation:
        print("🛑 Загрузка остановлена по запросу пользователя")
        return None
    
    all_models_config = get_all_models_config()
    
    if not model_name or model_name not in all_models_config:
        raise ValueError(f"Unknown model: {model_name}")
    
    if model_name in loaded_models:
        print(f"✅ Модель '{model_name}' уже загружена")
        return loaded_models[model_name]
    
    config = all_models_config[model_name]
    
    print(f"📥 Загрузка модели: {model_name}")
    
    def download_hf_file(hf_path, file_type, progress_stage):
        if hf_path.startswith('hf://'):
            if progress_callback:
                progress_callback(f"📥 Загрузка {file_type} из Hugging Face...", progress_stage)
            
            try:
                repo_id, filename = parse_hf_path(hf_path)
                if not repo_id or not filename:
                    raise ValueError(f"Invalid HF path format: {hf_path}")
                
                print(f"Downloading {file_type} from HF: {repo_id}/{filename}")

                downloaded_path = hf_hub_download(
                    repo_id=repo_id, 
                    filename=filename,
                    repo_type="model",
                    token=HF_TOKEN
                )
                print(f"✅ {file_type} downloaded to: {downloaded_path}")
                return downloaded_path
                
            except Exception as e:
                print(f"❌ Failed to download {file_type} from HF: {e}")
                if "404" in str(e):
                    raise FileNotFoundError(f"Файл {filename} не найден в репозитории {repo_id}")
                elif "401" in str(e):
                    if HF_TOKEN is None:
                        raise PermissionError(f"Нет доступа к репозиторию {repo_id}. Требуется авторизация!")
                    else:
                        raise PermissionError(f"Нет доступа к репозиторию {repo_id}. Проверьте токен и права доступа.")
                else:
                    raise e
        else:
            if not os.path.exists(hf_path):
                raise FileNotFoundError(f"{file_type} file not found: {hf_path}")
            return hf_path
    
    if config['repo'] == "local":
        if progress_callback:
            progress_callback("🔍 Обработка локальной модели...", 0.1)

        if stop_generation:
            print("🛑 Остановка на этапе поиска модели")
            return None
        
        model_path = config['model_file']
        vocab_path = config['vocab_file']
        
        try:
            model_path = download_hf_file(model_path, "model", 0.2)
            vocab_path = download_hf_file(vocab_path, "vocab", 0.3)
            
        except Exception as e:
            error_msg = f"❌ Ошибка загрузки файлов для локальной модели: {str(e)}"
            print(error_msg)
            raise e
            
        print(f"📁 Loading local model from: {model_path}")
        
    else:
        if progress_callback:
            progress_callback("🔍 Поиск модели в HuggingFace...", 0.1)
        
        if stop_generation:
            print("🛑 Остановка на этапе поиска модели в HF")
            return None
        
        check_cuda_availability()

        if stop_generation:
            print("🛑 Остановка после проверки CUDA")
            return None
        
        if progress_callback:
            progress_callback("📥 Загрузка модели...", 0.3)
        
        if stop_generation:
            print("🛑 Остановка перед загрузкой файлов")
            return None
        
        try:
            model_path = hf_hub_download(
                repo_id=config['repo'], 
                filename=config['model_file'],
                repo_type="model"
            )
            vocab_path = hf_hub_download(
                repo_id=config['repo'], 
                filename=config['vocab_file'],
                repo_type="model"
            )
            print(f"✅ Model downloaded to: {model_path}")
            print(f"✅ Vocab downloaded to: {vocab_path}")
            
        except Exception as e:
            print("❌ hf_hub_download failed:", e)
            
            if progress_callback:
                progress_callback("📦 Резервное скачивание полной модели...", 0.5)
            
            if stop_generation:
                print("🛑 Остановка перед резервной стратегией")
                return None
            
            try:
                local_dir = f"cache_{config['repo'].replace('/', '_')}"
                snapshot_dir = snapshot_download(
                    repo_id=config['repo'], 
                    cache_dir=None, 
                    local_dir=local_dir,
                    repo_type="model",
                    allow_patterns=[config['model_file'], config['vocab_file']]
                )
                
                possible_model = os.path.join(snapshot_dir, config['model_file'])
                possible_vocab = os.path.join(snapshot_dir, config['vocab_file'])
                
                if os.path.exists(possible_model):
                    model_path = possible_model
                    print(f"✅ Model found in snapshot: {model_path}")
                else:
                    print(f"❌ Model file not found in snapshot: {possible_model}")
                    
                if os.path.exists(possible_vocab):
                    vocab_path = possible_vocab
                    print(f"✅ Vocab found in snapshot: {vocab_path}")
                else:
                    print(f"❌ Vocab file not found in snapshot: {possible_vocab}")
                    
            except Exception as snapshot_error:
                print("❌ snapshot_download also failed:", snapshot_error)
                combined_error = f"Original error: {e}\nSnapshot error: {snapshot_error}"
                raise Exception(f"Failed to download model: {combined_error}")
    
    if stop_generation:
        print("🛑 Generation stopped during file download")
        return None
    
    if not model_path or not os.path.exists(model_path):
        raise FileNotFoundError(f"❌ Model file not found: {model_path}")
    if not vocab_path or not os.path.exists(vocab_path):
        raise FileNotFoundError(f"❌ Vocab file not found: {vocab_path}")
    
    if stop_generation:
        print("🛑 Остановка перед загрузкой в память")
        return None

    if progress_callback:
        progress_callback("🔄 Загрузка модели в память...", 0.7)
    
    try:
        print(f"🤖 Loading model architecture from: {model_path}")
        print(f"📖 Using vocab file: {vocab_path}")
        
        model = load_model(DiT, config['model_cfg'], model_path, vocab_file=vocab_path)
        print("✅ Model architecture loaded successfully")
        
    except Exception as e:
        error_msg = f"❌ Failed to load model architecture: {str(e)}"
        print(error_msg)
        print(f"Model config: {config['model_cfg']}")
        raise e

    if stop_generation:
        print("🛑 Generation stopped during model loading")
        return None
        
    if stop_generation:
        print("🛑 Остановка перед переносом на GPU")
        return None
        
    if progress_callback:
        progress_callback("🚀 Перенос модели на GPU...", 0.9)
    
    try:
        device = torch.device("cuda")
        model.to(device)
        print(f"✅ Model moved to CUDA: {device}")
        
        if next(model.parameters()).is_cuda:
            print("✅ Model parameters are on GPU")
        else:
            print("⚠️ Warning: Model parameters might not be on GPU")
            
    except Exception as e:
        error_msg = f"❌ Failed to move model to GPU: {str(e)}"
        print(error_msg)
        raise e

    if stop_generation:
        print("🛑 Остановка перед сохранением в кэш")
        return None

    loaded_models[model_name] = model
    current_model_name = model_name

    if progress_callback:
        progress_callback("✅ Модель загружена и готова!", 1.0)
    
    print(f"🎉 Model '{model_name}' successfully loaded and cached")
    print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    return model

def ensure_model(model_name, progress_callback=None):
    """Ensure model is loaded with progress tracking"""
    global loaded_models
    
    if not model_name:
        raise ValueError("Model name must be specified")
    
    all_models_config = get_all_models_config()
    
    if model_name not in all_models_config:
        raise ValueError(f"Unknown model: {model_name}")
    
    if model_name in loaded_models:
        return loaded_models[model_name]
    
    return load_model_with_progress(model_name, progress_callback)

def ensure_vocoder():
    global loaded_vocoder
    if loaded_vocoder is not None:
        return loaded_vocoder

    check_cuda_availability()

    print("⏳ Loading vocoder...")
    
    try:
        loaded_vocoder = load_vocoder()
        device = torch.device("cuda")
        loaded_vocoder.to(device)
        print("✅ Vocoder loaded successfully")
        return loaded_vocoder
    except Exception as e:
        print(f"❌ Vocoder loading failed: {e}")
        raise e

def stop_generation_process():
    """Остановка генерации - устанавливает глобальный флаг"""
    global stop_generation, generation_event, stop_requested
    
    print("🛑 Пользователь запросил остановку генерации...")
    stop_generation = True
    stop_requested = True
    generation_event.set()
    
    global generation_thread
    if generation_thread and generation_thread.is_alive():
        print("⚠️ Пытаемся прервать поток генерации...")
    
    torch.cuda.empty_cache()
    gc.collect()
    
    print("✅ Флаг остановки установлен, очищена память CUDA")
    return "🛑 Запрос на остановку отправлен..."

def reset_generation_flags():
    """Сброс флагов генерации перед началом новой генерации"""
    global stop_generation, generation_event, stop_requested
    stop_generation = False
    stop_requested = False
    generation_event.clear()
    print("🔄 Флаги генерации сброшены")

def check_stop_generation():
    """Проверка флага остановки (вызывается из функций генерации)"""
    global stop_generation, generation_event
    
    if stop_generation:
        print("🛑 Обнаружен запрос остановки!")
        raise KeyboardInterrupt("Генерация остановлена пользователем")
    
    if generation_event.is_set():
        print("🛑 Обнаружено событие остановки!")
        raise KeyboardInterrupt("Генерация остановлена пользователем")
    
    return False

# ================ НОВЫЕ ФУНКЦИИ ДЛЯ АНАЛИЗА И НОРМАЛИЗАЦИИ АУДИО ================

def analyze_audio_loudness(audio_path):
    """
    Анализирует громкость аудиофайла и возвращает рекомендации
    
    Возвращает:
    - среднюю громкость (dBFS)
    - пиковую громкость (dBFS)
    - динамический диапазон
    - рекомендации
    - визуализацию уровня громкости
    """
    if not audio_path or not os.path.exists(audio_path):
        return None, None, None, "❌ Аудиофайл не найден", ""
    
    try:
        # Загрузка аудио
        audio = AudioSegment.from_file(audio_path)
        
        # Рассчет громкости
        loudness = audio.dBFS  # Средняя громкость в dBFS
        max_loudness = audio.max_dBFS  # Пиковая громкость
        
        # Рассчет RMS для более точного анализа
        samples = np.array(audio.get_array_of_samples())
        if audio.channels == 2:
            samples = samples.reshape((-1, 2)).mean(axis=1)
        
        rms = np.sqrt(np.mean(samples.astype(np.float64)**2))
        rms_db = 20 * np.log10(rms / (2**15)) if rms > 0 else -100
        
        # Динамический диапазон
        dynamic_range = max_loudness - loudness if max_loudness > -100 and loudness > -100 else 0
        
        # Анализ громкости
        loudness_status = "🔴 СЛИШКОМ ТИХО"
        loudness_emoji = "🔴"
        recommendation = "Требуется нормализация: увеличение громкости на 10-15 dB"
        
        if loudness > -20:
            loudness_status = "🟢 ОПТИМАЛЬНО"
            loudness_emoji = "🟢"
            recommendation = "Громкость в норме. Можно слегка увеличить для большего присутствия (+3 dB)"
        elif loudness > -25:
            loudness_status = "🟡 НОРМАЛЬНО"
            loudness_emoji = "🟡"
            recommendation = "Громкость приемлемая. Рекомендуется легкая нормализация (+5 dB)"
        elif loudness > -30:
            loudness_status = "🟠 ТИХО"
            loudness_emoji = "🟠"
            recommendation = "Запись тихая. Рекомендуется нормализация (+10 dB)"
        
        # Анализ пиков
        peak_status = "🟢 НОРМА"
        if max_loudness > -1:
            peak_status = "🔴 ПЕРЕГРУЗКА!"
            recommendation += " ВНИМАНИЕ: Обнаружены клиппинг (перегрузка)!"
        elif max_loudness > -3:
            peak_status = "🟠 БЛИЗКО К ПРЕДЕЛУ"
            recommendation += " Будьте осторожны: пики близки к пределу."
        
        # Создание визуализации уровня громкости
        loudness_bar = create_loudness_visualization(loudness, max_loudness)
        
        # Формирование подробного отчета
        analysis_report = f"""
        <div style="background: #f8f9fa; padding: 20px; border-radius: 10px; border-left: 5px solid {'#28a745' if loudness_emoji == '🟢' else '#ffc107' if loudness_emoji == '🟡' else '#fd7e14' if loudness_emoji == '🟠' else '#dc3545'};">
            <h3>📊 Анализ громкости</h3>
            
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin: 15px 0;">
                <div style="background: white; padding: 10px; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    <strong>Средняя громкость:</strong><br>
                    <span style="font-size: 24px; font-weight: bold; color: {'#28a745' if loudness_emoji == '🟢' else '#ffc107' if loudness_emoji == '🟡' else '#fd7e14' if loudness_emoji == '🟠' else '#dc3545'}">
                        {loudness:.1f} dBFS
                    </span><br>
                    <strong>Статус:</strong> {loudness_status}
                </div>
                
                <div style="background: white; padding: 10px; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    <strong>Пиковая громкость:</strong><br>
                    <span style="font-size: 24px; font-weight: bold; color: {'#28a745' if max_loudness < -3 else '#ffc107' if max_loudness < -1 else '#dc3545'}">
                        {max_loudness:.1f} dBFS
                    </span><br>
                    <strong>Статус:</strong> {peak_status}
                </div>
            </div>
            
            <div style="margin: 15px 0;">
                <strong>Визуализация уровня:</strong><br>
                {loudness_bar}
            </div>
            
            <div style="background: white; padding: 15px; border-radius: 5px; margin: 15px 0; border-left: 4px solid #007bff;">
                <strong>💡 Рекомендации:</strong><br>
                {recommendation}
            </div>
            
            <div style="font-size: 12px; color: #6c757d; margin-top: 10px;">
                📈 <strong>Технические детали:</strong><br>
                • Динамический диапазон: {dynamic_range:.1f} dB<br>
                • RMS уровень: {rms_db:.1f} dBFS<br>
                • Длительность: {len(audio)/1000:.1f} сек<br>
                • Частота: {audio.frame_rate} Гц<br>
                • Каналы: {audio.channels}
            </div>
        </div>
        """
        
        return loudness, max_loudness, dynamic_range, analysis_report, loudness_bar
        
    except Exception as e:
        print(f"❌ Ошибка анализа аудио: {e}")
        return None, None, None, f"❌ Ошибка анализа: {str(e)[:100]}", ""

def create_loudness_visualization(loudness, max_loudness):
    """Создает визуализацию уровня громкости"""
    # Нормализуем значения для отображения
    loudness_norm = max(0, min(100, (loudness + 60) * 1.67))  # -60dB = 0%, 0dB = 100%
    max_norm = max(0, min(100, (max_loudness + 60) * 1.67))
    
    # Определяем цветовые зоны
    def get_color(value):
        if value < 25:
            return "#dc3545"  # Красный - слишком тихо
        elif value < 50:
            return "#fd7e14"  # Оранжевый - тихо
        elif value < 75:
            return "#ffc107"  # Желтый - нормально
        else:
            return "#28a745"  # Зеленый - оптимально
    
    loudness_color = get_color(loudness_norm)
    peak_color = "#6f42c1" if max_norm > 95 else get_color(max_norm)
    
    # Создаем HTML для визуализации
    visualization = f"""
    <div style="width: 100%; height: 40px; background: linear-gradient(to right, #dc3545 0%, #fd7e14 25%, #ffc107 50%, #28a745 75%, #28a745 100%); border-radius: 5px; position: relative; margin: 10px 0;">
        <!-- Средний уровень -->
        <div style="position: absolute; left: {loudness_norm}%; top: 0; bottom: 0; width: 4px; background: {loudness_color}; border-radius: 2px; transform: translateX(-50%);">
            <div style="position: absolute; bottom: 100%; left: 50%; transform: translateX(-50%); white-space: nowrap; font-size: 12px; font-weight: bold; color: {loudness_color};">
                Сред: {loudness:.1f}dB
            </div>
        </div>
        
        <!-- Пиковый уровень -->
        <div style="position: absolute; left: {max_norm}%; top: 0; bottom: 0; width: 4px; background: {peak_color}; border-radius: 2px; transform: translateX(-50%);">
            <div style="position: absolute; top: 100%; left: 50%; transform: translateX(-50%); white-space: nowrap; font-size: 12px; font-weight: bold; color: {peak_color};">
                Пик: {max_loudness:.1f}dB
            </div>
        </div>
        
        <!-- Разметка -->
        <div style="position: absolute; left: 0; right: 0; top: 50%; transform: translateY(-50%); display: flex; justify-content: space-between; padding: 0 5px; font-size: 10px; color: #666;">
            <span>-60dB</span>
            <span>-30dB</span>
            <span>0dB</span>
        </div>
    </div>
    
    <div style="display: flex; justify-content: space-between; font-size: 11px; color: #6c757d; margin-top: 5px;">
        <span>🔴 Слишком тихо</span>
        <span>🟠 Тихо</span>
        <span>🟡 Норма</span>
        <span>🟢 Оптимально</span>
    </div>
    """
    
    return visualization

def normalize_audio_enhanced(audio_path, target_loudness=-16.0, headroom=1.0, audio_format=None):
    """
    Улучшенная нормализация аудио с сохранением в тот же формат
    
    Args:
        audio_path: путь к исходному аудиофайлу
        target_loudness: целевая громкость в dBFS (стандарт: -16 dBFS для подкастов, -14 для музыки)
        headroom: запас до клиппинга в dB
        audio_format: формат для сохранения (если None, определяется из исходного файла)
    
    Returns:
        путь к нормализованному файлу
    """
    if not audio_path or not os.path.exists(audio_path):
        return None, "❌ Аудиофайл не найден"
    
    try:
        print(f"🎚️ Начинаем нормализацию аудио: {audio_path}")
        
        # Определяем формат
        if audio_format is None:
            audio_format = os.path.splitext(audio_path)[1].lower().replace('.', '')
            if audio_format not in ['wav', 'mp3', 'ogg', 'flac']:
                audio_format = 'wav'  # формат по умолчанию
        
        # Загрузка аудио
        audio = AudioSegment.from_file(audio_path)
        
        # Анализируем текущую громкость
        current_loudness = audio.dBFS
        max_loudness = audio.max_dBFS
        print(f"📊 Текущая громкость: {current_loudness:.1f} dBFS")
        print(f"📊 Пиковая громкость: {max_loudness:.1f} dBFS")
        
        # Рассчитываем необходимую коррекцию
        needed_gain = target_loudness - current_loudness
        
        # Учитываем запас до клиппинга
        if max_loudness + needed_gain > -headroom:
            # Ограничиваем усиление, чтобы избежать клиппинга
            safe_gain = (-headroom) - max_loudness
            if safe_gain < needed_gain:
                needed_gain = safe_gain
                print(f"⚠️ Ограничиваем усиление для избежания клиппинга: {needed_gain:.1f} dB")
        
        print(f"🎛️ Применяем усиление: {needed_gain:.1f} dB")
        
        # Применяем усиление
        if abs(needed_gain) > 0.5:  # Корректируем только если разница значительная
            normalized_audio = audio.apply_gain(needed_gain)
        else:
            print("ℹ️ Громкость уже близка к целевой, усиление не требуется")
            normalized_audio = audio
        
        # Дополнительная нормализация через pydub (компрессия мягких пиков)
        try:
            normalized_audio = normalize(normalized_audio)
            print("✅ Применена дополнительная нормализация")
        except:
            print("⚠️ Дополнительная нормализация не удалась, используем базовую")
        
        # Мягкое ограничение пиков (soft clipping)
        try:
            from pydub.effects import compress_dynamic_range
            normalized_audio = compress_dynamic_range(normalized_audio, threshold=-3.0, ratio=2.0)
            print("✅ Применена мягкая компрессия для контроля пиков")
        except:
            print("⚠️ Компрессия не доступна, пропускаем")
        
        # Сохранение нормализованного файла
        timestamp = int(time.time())
        base_name = os.path.splitext(os.path.basename(audio_path))[0]
        normalized_filename = f"{base_name}_normalized_{timestamp}.{audio_format}"
        normalized_path = os.path.join(os.path.dirname(audio_path), normalized_filename)
        
        # Настройки экспорта в зависимости от формата
        export_params = {}
        if audio_format == 'mp3':
            export_params = {'format': 'mp3', 'bitrate': '192k'}
        elif audio_format == 'ogg':
            export_params = {'format': 'ogg', 'bitrate': '192k'}
        elif audio_format == 'flac':
            export_params = {'format': 'flac'}
        else:  # wav
            export_params = {'format': 'wav'}
        
        normalized_audio.export(normalized_path, **export_params)
        
        # Проверяем результат
        normalized_audio_check = AudioSegment.from_file(normalized_path)
        final_loudness = normalized_audio_check.dBFS
        final_max = normalized_audio_check.max_dBFS
        
        print(f"✅ Нормализация завершена!")
        print(f"📊 Итоговая громкость: {final_loudness:.1f} dBFS")
        print(f"📊 Итоговый пик: {final_max:.1f} dBFS")
        print(f"💾 Файл сохранен: {normalized_path}")
        
        # Сохраняем в кэш
        global normalized_audio_cache
        normalized_audio_cache[audio_path] = normalized_path
        
        # Формируем отчет
        report = f"""
        <div style="background: #d4edda; padding: 15px; border-radius: 8px; border-left: 5px solid #28a745; margin: 10px 0;">
            <h4 style="margin-top: 0; color: #155724;">✅ Нормализация успешно завершена!</h4>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px;">
                <div style="background: white; padding: 8px; border-radius: 5px;">
                    <strong>Было:</strong><br>
                    Средняя: {current_loudness:.1f} dBFS<br>
                    Пиковая: {max_loudness:.1f} dBFS
                </div>
                <div style="background: white; padding: 8px; border-radius: 5px;">
                    <strong>Стало:</strong><br>
                    Средняя: {final_loudness:.1f} dBFS<br>
                    Пиковая: {final_max:.1f} dBFS
                </div>
            </div>
            <div style="margin-top: 10px; font-size: 14px;">
                <strong>Усиление:</strong> {needed_gain:.1f} dB<br>
                <strong>Формат:</strong> {audio_format.upper()}<br>
                <strong>Файл:</strong> {normalized_filename}
            </div>
        </div>
        """
        
        return normalized_path, report
        
    except Exception as e:
        error_msg = f"❌ Ошибка нормализации: {str(e)}"
        print(f"Ошибка нормализации аудио: {traceback.format_exc()}")
        return None, error_msg

def get_normalized_audio_path(original_path):
    """Получить путь к нормализованной версии аудио из кэша"""
    global normalized_audio_cache
    return normalized_audio_cache.get(original_path)

# ================ КОНЕЦ НОВЫХ ФУНКЦИЙ ================

print("Loading RUNorm (text normalizer)...")
try:
    normalizer = RUNorm()
    normalizer.load(
        model_size="medium",
        device="cuda" if torch.cuda.is_available() else "cpu",
        workdir="./local_cache"
    )
    
    test_text = "13,8 миллиардов лет"
    try:
        result = normalizer.norm(test_text)
        print(f"✅ RUNorm.norm SUCCESS: '{test_text}' -> '{result}'")
        print("✅ RUNorm loaded successfully with 'norm' method")
    except Exception as e:
        print(f"❌ RUNorm.norm failed: {e}")
        class DummyNormalizer:
            def norm(self, text):
                return text
        normalizer = DummyNormalizer()
        print("⚠️ Using dummy normalizer due to error")
    
except Exception as e:
    print(f"❌ Failed to load RUNorm: {e}")
    print(f"🔍 RUNorm error details: {traceback.format_exc()}")
    
    class DummyNormalizer:
        def norm(self, text):
            return text
    normalizer = DummyNormalizer()
    print("⚠️ Using dummy normalizer due to load error")

print("Loading Silero Stress model...")
accentor = load_accentor()

test_word = "привет"
test_result = accentor(test_word)
print(f"✅ Silero Stress loaded. Тест: '{test_word}' -> '{test_result}'")

print("Loading Whisper ASR model (supports Russian and English)...")
try:
    from transformers import WhisperProcessor, WhisperForConditionalGeneration

    ASR_MODEL_NAME = "openai/whisper-medium"
    asr_processor = WhisperProcessor.from_pretrained(ASR_MODEL_NAME)
    asr_model = WhisperForConditionalGeneration.from_pretrained(ASR_MODEL_NAME)
    asr_model.to("cuda" if torch.cuda.is_available() else "cpu")
    print("Whisper model loaded. Supports Russian, English, and many other languages.")
except Exception as e:
    print(f"Failed to load Whisper model: {e}")
    asr_processor = None
    asr_model = None

def check_audio_file(audio_path):
    """Проверка аудиофайла"""
    if not audio_path:
        return gr.update(value="❌ Нет аудиофайла", visible=True)
    
    try:
        if not os.path.exists(audio_path):
            return gr.update(value="❌ Файл не существует", visible=True)
        
        valid_extensions = ('.wav', '.mp3', '.ogg', '.flac', '.m4a', '.aac')
        if not audio_path.lower().endswith(valid_extensions):
            return gr.update(value="❌ Неподдерживаемый формат", visible=True)
        
        try:
            audio = AudioSegment.from_file(audio_path)
            duration = len(audio) / 1000.0
            
            return gr.update(
                value=f"✅ Аудио файл корректен:<br>"
                      f"• Длительность: {duration:.1f} сек<br>"
                      f"• Частота: {audio.frame_rate} Гц<br>"
                      f"• Каналы: {audio.channels}<br>"
                      f"• Размер: {os.path.getsize(audio_path) / 1024:.1f} KB",
                visible=True
            )
        except Exception as e:
            return gr.update(value=f"❌ Ошибка загрузки аудио: {str(e)[:50]}", visible=True)
            
    except Exception as e:
        return gr.update(value=f"❌ Ошибка: {str(e)[:50]}", visible=True)

def test_whisper_model():
    """Проверка работы Whisper модели перед запуском"""
    print("\n" + "="*60)
    print("🤖 ТЕСТИРОВАНИЕ WHISPER МОДЕЛИ")
    print("="*60)
    
    if asr_processor is None or asr_model is None:
        print("❌ Whisper модель не загружена")
        return False
    
    try:
        sample_rate = 16000
        duration = 2.0
        t = np.linspace(0, duration, int(sample_rate * duration))
        test_audio = 0.5 * np.sin(2 * np.pi * 440 * t)
        
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
            test_file = f.name
            sf.write(test_file, test_audio, sample_rate)
        
        print("🎵 Создан тестовый аудиофайл...")
        
        transcription = transcribe_audio(test_file)
        
        try:
            os.unlink(test_file)
        except:
            pass
        
        if transcription:
            print(f"✅ Whisper работает! Тестовая транскрипция: '{transcription}'")
            return True
        else:
            print("❌ Whisper вернул пустую транскрипция")
            return False
            
    except Exception as e:
        print(f"❌ Ошибка тестирования Whisper: {e}")
        return False

def check_all_models_before_launch():
    """Проверка всех моделей перед запуском интерфейса"""
    print("\n" + "="*60)
    print("🚀 ПРЕДВАРИТЕЛЬНАЯ ПРОВЕРКА МОДЕЛЕЙ")
    print("="*60)
    
    try:
        check_cuda_availability()
        print("✅ CUDA доступен")
    except RuntimeError as e:
        print(f"❌ CUDA не доступен: {e}")
        return False
    
    try:
        test_text = "13,8 миллиардов лет"
        result = normalizer.norm(test_text)
        print(f"✅ RUNorm работает: '{test_text}' -> '{result}'")
    except Exception as e:
        print(f"⚠️ RUNorm не работает: {e}")
    
    try:
        test_word = "привет"
        result = accentor(test_word)
        print(f"✅ Silero Stress работает: '{test_word}' -> '{result}'")
    except Exception as e:
        print(f"⚠️ Silero Stress не работает: {e}")
    
    whisper_ok = test_whisper_model()
    
    if not whisper_ok:
        print("⚠️ ВНИМАНИЕ: Whisper может не работать корректно!")
        print("   Аудио транскрипция будет отключена.")
    
    print("="*60)
    return True

def transcribe_audio(audio_path):
    if not audio_path or asr_processor is None or asr_model is None:
        return ""

    try:
        waveform, sample_rate = torchaudio.load(audio_path)

        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resampler(waveform)

        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        input_audio = waveform.squeeze().numpy()

        inputs = asr_processor(
            input_audio,
            sampling_rate=16000,
            return_tensors="pt",
        )

        input_features = inputs.input_features

        current_length = input_features.shape[-1]
        if current_length < 3000:
            pad_length = 3000 - current_length
            input_features = torch.nn.functional.pad(input_features, (0, pad_length), mode='constant', value=0)
        elif current_length > 3000:
            input_features = input_features[..., :3000]

        print("Input features shape:", input_features.shape)

        input_features = input_features.to(asr_model.device)

        with torch.no_grad():
            generated_ids = asr_model.generate(
                input_features=input_features,
                language=None,
                task="transcribe",
            )

        transcription = asr_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return transcription.strip().capitalize() + "."

    except Exception as e:
        print("ASR transcription failed:", e)
        return "Ошибка распознавания."

def validate_and_transcribe_audio(audio_path):
    """Валидация аудиофайла (без автоматической транскрипции)"""
    if not audio_path:
        return None, "", gr.update(value="", visible=False)
    
    try:
        return audio_path, "", gr.update(value="", visible=False)
        
    except Exception as e:
        print("Validation failed:", e)
        return None, "", gr.update(value="⚠️ **Ошибка при проверке аудио**", visible=True)

def manual_transcribe_audio(audio_path):
    """Ручная транскрипция аудио"""
    if not audio_path:
        return "", gr.update(value="❌ Нет аудиофайла", visible=True)
    
    if asr_processor is None or asr_model is None:
        return "", gr.update(value="❌ Whisper не загружен", visible=True)
    
    try:
        transcription = transcribe_audio(audio_path)
        if transcription:
            return transcription, gr.update(value="✅ Транскрипция завершена", visible=True)
        else:
            return "", gr.update(value="❌ Не удалось транскрибировать", visible=True)
    except Exception as e:
        print(f"Ошибка ручной транскрипции: {e}")
        return "", gr.update(value=f"❌ Ошибка: {str(e)[:50]}", visible=True)

def process_text_with_accent(text):
    """Обработка текста с добавлением ударений через silero-stress с сохранением букв 'ё'"""
    if not text or not text.strip():
        print("❌ Пустой текст для обработки")
        return text

    if '+' in text:
        print("✅ Текст уже содержит ударения, пропускаем обработку")
        return text

    try:
        print(f"🎯 Начинаем обработку текста: '{text}'")
        
        words_and_delimiters = re.split(r'(\W+)', text)
        result_parts = []
        
        print(f"🔍 Найдено {len(words_and_delimiters)} частей")
        
        for i, part in enumerate(words_and_delimiters):
            if part.strip() and re.match(r'^\w+$', part):
                print(f"🔤 Обработка слова {i+1}: '{part}'")
                try:
                    processed_word = accentor(part)
                    print(f"✅ Слово '{part}' -> '{processed_word}'")
                    
                    processed_word_list = list(processed_word)
                    original_word = part
                    
                    for j, orig_char in enumerate(original_word):
                        if j < len(processed_word_list):
                            if (orig_char in ['ё', 'Ё'] and 
                                processed_word_list[j].lower() == 'е'):
                                processed_word_list[j] = orig_char
                                print(f"🔄 Восстановлена буква '{orig_char}' на позиции {j}")
                    
                    processed_word = ''.join(processed_word_list)
                    print(f"🔄 После восстановления ё/Ё: '{processed_word}'")
                    
                    if part and processed_word:
                        if part[0].isupper():
                            processed_word = processed_word[0].upper() + processed_word[1:]
                            print(f"🔠 Восстановлен верхний регистр: '{processed_word}'")
                        else:
                            processed_word = processed_word[0].lower() + processed_word[1:]
                            print(f"🔡 Восстановлен нижний регистр: '{processed_word}'")
                        
                        result_parts.append(processed_word)
                    else:
                        print(f"⚠️ Пропускаем слово '{part}'")
                        result_parts.append(part)
                except Exception as e:
                    print(f"❌ Silero stress failed for word '{part}': {e}")
                    result_parts.append(part)
            else:
                if part.strip():
                    print(f"📎 Не-слово: '{part}'")
                result_parts.append(part)
        
        final_result = ''.join(result_parts)
        final_result = final_result.replace('[', '').replace(']', '')

        print(f"🎯 Финальный текст с ударениями и (без []): '{final_result}'")
        print(f"📊 Сравнение: Исходный: '{text}' -> Обработанный: '{final_result}'")
        return final_result
        
    except Exception as e:
        print(f"❌ Text processing failed: {e}")
        print(f"🔍 Traceback: {traceback.format_exc()}")
        return text

def normalize_gen_text(gen_text):
    """Нормализация только текста для генерации (silero-stress)"""
    if gen_text and isinstance(gen_text, str):
        if any(bad in gen_text for bad in ["/tmp/", ".wav", ".mp3", ".ogg", ".flac", "/home/", "/root/"]):
            gen_text = ""
    
    print("=" * 60)
    print("🔄 НОРМАЛИЗАЦИЯ ТЕКСТА ДЛЯ ГЕНЕРАЦИИ")
    print(f"📖 Текст до обработки: '{gen_text}'")
    
    contains_digits = any(char.isdigit() for char in gen_text) if gen_text else False
    
    if gen_text and contains_digits:
        print("🔢 Текст содержит цифры, запускаем RUNorm...")
        print("🔧 Шаг 1: Нормализация через RUNorm...")
        try:
            normalized_text = normalizer.norm(gen_text)
            print(f"📝 После RUNorm: '{normalized_text}'")
        except Exception as norm_error:
            print(f"❌ Ошибка RUNorm: {norm_error}")
            normalized_text = gen_text
    else:
        if gen_text and not contains_digits:
            print("🔤 Текст не содержит цифр, пропускаем RUNorm...")
        normalized_text = gen_text if gen_text else ""
    
    if normalized_text:
        print("🔧 Шаг 2: Обработка ударений...")
        processed_gen = process_text_with_accent(normalized_text)
        print(f"✅ Текст после обработки: '{processed_gen}'")
    else:
        processed_gen = ""
    
    print("=" * 60)
    
    return processed_gen

def normalize_ref_text(ref_text):
    """Нормализация референсного текста (RUNorm + silero-stress)"""
    if ref_text and isinstance(ref_text, str):
        if any(bad in ref_text for bad in ["/tmp/", ".wav", ".mp3", ".ogg", ".flac", "/home/", "/root/"]):
            ref_text = ""
    
    print("=" * 60)
    print("🔄 НОРМАЛИЗАЦИЯ РЕФЕРЕНСНОГО ТЕКСТА")
    print(f"📖 Референс текст до обработки: '{ref_text}'")
    
    contains_digits = any(char.isdigit() for char in ref_text) if ref_text else False
    
    if ref_text and contains_digits:
        print("🔢 Текст содержит цифры, запускаем RUNorm...")
        print("🔧 Шаг 1: Нормализация через RUNorm...")
        try:
            normalized_text = normalizer.norm(ref_text)
            print(f"📝 После RUNorm: '{normalized_text}'")
        except Exception as norm_error:
            print(f"❌ Ошибка RUNorm: {norm_error}")
            normalized_text = ref_text
    else:
        if ref_text and not contains_digits:
            print("🔤 Текст не содержит цифр, пропускаем RUNorm...")
        normalized_text = ref_text if ref_text else ""
    
    if normalized_text:
        print("🔧 Шаг 2: Обработка ударений...")
        processed_ref = process_text_with_accent(normalized_text)
        print(f"✅ Референс текст после обработки: '{processed_ref}'")
    else:
        processed_ref = ""
    
    print("=" * 60)
    return processed_ref

def normalize_ref_text_with_steps(ref_text):
    """Нормализация референсного текста с раздельными шагами RUNorm → Silero Stress"""
    if not ref_text or not ref_text.strip():
        return ref_text, "❌ Пустой текст", "❌ Пустой текст"
    
    print("=" * 60)
    print("🔄 НОРМАЛИЗАЦИЯ РЕФЕРЕНСНОГО ТЕКСТА (2 ШАГА)")
    print(f"📖 Исходный текст: '{ref_text}'")
    
    step1_result = ""
    step2_result = ""
    step1_status = ""
    
    try:
        contains_digits = any(char.isdigit() for char in ref_text)
        
        if contains_digits:
            print("🔢 Текст содержит цифры, запускаем RUNorm...")
            print("🔧 Шаг 1: Запуск RUNorm...")
            step1_result = normalizer.norm(ref_text)
            print(f"✅ RUNorm результат: '{step1_result}'")
            step1_status = f"✅ RUNorm (цифры найдены): '{step1_result}'"
        else:
            print("🔤 Текст не содержит цифр, пропускаем RUNorm...")
            step1_result = ref_text
            step1_status = "⏭️ RUNorm пропущен (нет цифр)"
        
        print("🔧 Шаг 2: Запуск Silero Stress...")
        step2_result = process_text_with_accent(step1_result)
        print(f"✅ Silero Stress результат: '{step2_result}'")

        step2_cleaned = step2_result.replace('[', '').replace(']', '')
        print(f"🧹 Удалены символы []: '{step2_result}' → '{step2_cleaned}'")

        print("=" * 60)
        print(f"🎯 Итог: '{ref_text}' → '{step2_cleaned}'")
        print(f"📏 Статистика:")
        print(f"   Исходный текст: {len(ref_text)} символов")
        print(f"   После обработки: {len(step2_cleaned)} символов")

        return step2_cleaned, step1_status, f"✅ Silero Stress: '{step2_cleaned}'"
        
    except Exception as e:
        print(f"❌ Ошибка нормализации: {e}")
        traceback.print_exc()
        return ref_text, f"❌ Ошибка: {str(e)[:50]}", "❌ Ошибка на шаге 2"

def normalize_gen_text_with_steps(gen_text):
    """Нормализация текста генерации с раздельными шагами RUNorm → Silero Stress"""
    if not gen_text or not gen_text.strip():
        return gen_text, "❌ Пустой текст", "❌ Пустой текст"
    
    print("=" * 60)
    print("🔄 НОРМАЛИЗАЦИЯ ТЕКСТА ГЕНЕРАЦИИ (2 ШАГА)")
    print(f"📖 Исходный текст: '{gen_text}'")
    
    step1_result = ""
    step2_result = ""
    step1_status = ""
    
    try:
        contains_digits = any(char.isdigit() for char in gen_text)
        
        if contains_digits:
            print("🔢 Текст содержит цифры, запускаем RUNorm...")
            print("🔧 Шаг 1: Запуск RUNorm...")
            
            try:
                step1_result = normalizer.norm(gen_text)
                print(f"✅ RUNorm результат: '{step1_result}'")
                step1_status = f"✅ RUNorm (цифры найдены): '{step1_result}'"
                
            except Exception as e:
                error_msg = str(e)
                print(f"⚠️ RUNorm вернул ошибку: {error_msg}")
                
                if "'token_type_ids'" in error_msg:
                    print("🛠️ Обнаружена ошибка token_type_ids, используем patch для RUNorm...")
                    
                    try:
                        if hasattr(normalizer, 'tokenizer'):
                            tokenizer = normalizer.tokenizer
                            model = normalizer.model
                            device = normalizer.device
                            
                            def patched_norm(text):
                                inputs = tokenizer(
                                    text, 
                                    return_tensors="pt", 
                                    padding=True,
                                    truncation=True,
                                    max_length=512
                                )
                                
                                if "token_type_ids" in inputs:
                                    del inputs["token_type_ids"]
                                
                                inputs = {k: v.to(device) for k, v in inputs.items()}
                                
                                with torch.no_grad():
                                    outputs = model.generate(
                                        **inputs,
                                        max_length=512,
                                        num_beams=5,
                                        early_stopping=True,
                                        no_repeat_ngram_size=3
                                    )
                                
                                decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
                                return decoded[0] if decoded else text
                            
                            normalizer.norm = patched_norm
                            
                            step1_result = normalizer.norm(gen_text)
                            print(f"✅ RUNorm (исправленный) результат: '{step1_result}'")
                            step1_status = f"✅ RUNorm (исправлен, цифры найдены): '{step1_result}'"
                            
                        else:
                            step1_result = simple_digit_replacement(gen_text)
                            print(f"🔄 Используем простую замену цифр: '{step1_result}'")
                            step1_status = f"⚠️ RUNorm ошибка, простая замена: '{step1_result}'"
                            
                    except Exception as patch_error:
                        print(f"⚠️ Не удалось исправить RUNorm: {patch_error}")
                        step1_result = simple_digit_replacement(gen_text)
                        step1_status = f"⚠️ RUNorm неисправен, ручная замена: '{step1_result}'"
                        
                else:
                    step1_result = simple_digit_replacement(gen_text)
                    step1_status = f"⚠️ RUNorm ошибка, ручная замена: '{step1_result}'"
        else:
            print("🔤 Текст не содержит цифр, пропускаем RUNorm...")
            step1_result = gen_text
            step1_status = "⏭️ RUNorm пропущен (нет цифр)"
        
        print("🔧 Шаг 2: Запуск Silero Stress...")
        step2_result = process_text_with_accent(step1_result)
        print(f"✅ Silero Stress результат: '{step2_result}'")

        step2_cleaned = step2_result.replace('[', '').replace(']', '')
        print(f"🧹 Удалены символы []: '{step2_result}' → '{step2_cleaned}'")

        print("=" * 60)
        print(f"🎯 Итог: '{gen_text}' → '{step2_cleaned}'")

        return step2_cleaned, step1_status, f"✅ {step2_cleaned}"
        
    except Exception as e:
        print(f"❌ Ошибка нормализации: {e}")
        traceback.print_exc()
        return gen_text, f"❌ Ошибка: {str(e)[:50]}", "❌ Ошибка на шаге 2"

def simple_digit_replacement(text):
    """Простая замена цифр на слова (fallback)"""
    digit_map = {
        '0': 'ноль',
        '1': 'один',
        '2': 'два', 
        '3': 'три',
        '4': 'четыре',
        '5': 'пять',
        '6': 'шесть',
        '7': 'семь',
        '8': 'восемь',
        '9': 'девять'
    }
    
    import re
    
    def replace_single_digit(match):
        num = match.group()
        return digit_map.get(num, num)
    
    result = re.sub(r'\b\d\b', replace_single_digit, text)
    
    return result

def patch_runorm_on_startup():
    """Исправить RUNorm при запуске программы"""
    global normalizer
    
    print("\n🛠️ Проверка и исправление RUNorm...")
    
    try:
        test_text = "13,8 миллиардов лет"
        print(f"🧪 Тест RUNorm с текстом: '{test_text}'")
        
        try:
            result = normalizer.norm(test_text)
            print(f"✅ RUNorm работает нормально: '{result}'")
            
        except Exception as e:
            error_msg = str(e)
            print(f"⚠️ RUNorm ошибка: {error_msg}")
            
            if "'token_type_ids'" in error_msg:
                print("🔄 Пытаемся исправить RUNorm...")
                
                if hasattr(normalizer, 'tokenizer') and hasattr(normalizer, 'model'):
                    
                    original_norm = normalizer.norm
                    
                    def patched_norm_final(text):
                        try:
                            return original_norm(text)
                        except Exception as inner_e:
                            if "'token_type_ids'" in str(inner_e):
                                tokenizer = normalizer.tokenizer
                                model = normalizer.model
                                device = normalizer.device
                                
                                inputs = tokenizer(
                                    text, 
                                    return_tensors="pt", 
                                    padding=True,
                                    truncation=True,
                                    max_length=512
                                )
                                
                                if "token_type_ids" in inputs:
                                    del inputs["token_type_ids"]
                                
                                inputs = {k: v.to(device) for k, v in inputs.items()}
                                
                                with torch.no_grad():
                                    outputs = model.generate(
                                        **inputs,
                                        max_length=512,
                                        num_beams=5,
                                        early_stopping=True
                                    )
                                
                                decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
                                return decoded[0] if decoded else text
                            else:
                                raise inner_e
                    
                    normalizer.norm = patched_norm_final
                    
                    try:
                        result = normalizer.norm(test_text)
                        print(f"✅ RUNorm исправлен и работает: '{result}'")
                    except:
                        print("⚠️ Не удалось исправить RUNorm, используем заглушку")
                        class DummyNormalizer:
                            def norm(self, text):
                                return text
                        normalizer = DummyNormalizer()
                        
                else:
                    print("⚠️ Не удалось получить доступ к компонентам RUNorm, используем заглушку")
                    class DummyNormalizer:
                        def norm(self, text):
                            return text
                    normalizer = DummyNormalizer()
            else:
                print("⚠️ Неизвестная ошибка RUNorm, используем заглушку")
                class DummyNormalizer:
                    def norm(self, text):
                        return text
                normalizer = DummyNormalizer()
                
    except Exception as e:
        print(f"❌ Ошибка при исправлении RUNorm: {e}")
        class DummyNormalizer:
            def norm(self, text):
                return text
        normalizer = DummyNormalizer()

print("\n" + "="*60)
print("🔧 ПОДГОТОВКА МОДЕЛЕЙ")
print("="*60)

patch_runorm_on_startup()

test_text = "Тест 123"
print(f"\n🧪 Финальный тест нормализации: '{test_text}'")
try:
    normalized = normalize_gen_text_with_steps(test_text)
    print(f"✅ Нормализация работает: '{normalized[0]}'")
except Exception as e:
    print(f"❌ Нормализация не работает: {e}")

def get_current_seed_display():
    """Получить текущее значение seed для отображения"""
    global last_seed
    if last_seed == -1:
        last_seed = np.random.randint(0, 2**31 - 1)
        print(f"🎲 Инициализирован начальный seed: {last_seed}")
    return last_seed

def update_model_loading_status(model_name):
    """Обновление статуса загрузки модели"""
    global loaded_models
    
    if not model_name:
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)
    
    all_models_config = get_all_models_config()
    
    if model_name not in all_models_config:
        return gr.update(value=f"<div class='model-status model-error'>❌ Модель {model_name} не найдена</div>", visible=True)
    
    if model_name in loaded_models:
        model_type = "🔧 Локальная" if all_models_config[model_name]['repo'] == "local" else "🌐 HuggingFace"
        return gr.update(value=f"<div class='model-status model-loaded'>✅ {model_name} ({model_type}) загружена</div>", visible=True)
    else:
        model_type = "🔧 Локальная" if all_models_config[model_name]['repo'] == "local" else "🌐 HuggingFace"
        return gr.update(value=f"<div class='model-status model-loading'>🔄 {model_name} ({model_type}) не загружена</div>", visible=True)

def load_model_with_status(model_name, progress=gr.Progress()):
    """Загрузка модели с обновлением статуса"""
    global loaded_models
    
    if not model_name:
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)
    
    try:
        check_stop_generation()
    except KeyboardInterrupt:
        print("🛑 Загрузка модели остановлена пользователем")
        return gr.update(value="<div class='model-status model-error'>🛑 Загрузка остановлена</div>", visible=True)
    
    if model_name in loaded_models:
        return update_model_loading_status(model_name)
    
    try:
        check_stop_generation()
        
        progress(0.1, desc="🔄 Загрузка модели...")
        
        def progress_callback_with_stop(msg, p):
            try:
                check_stop_generation()
            except KeyboardInterrupt:
                print(f"🛑 Остановка на этапе: {msg}")
                raise
            
            progress(p, desc=msg)
        
        model = load_model_with_progress(model_name, progress_callback_with_stop)
        
        if model:
            return update_model_loading_status(model_name)
        else:
            return gr.update(value="<div class='model-status model-error'>❌ Загрузка отменена</div>", visible=True)
            
    except KeyboardInterrupt:
        print("🛑 Загрузка модели прервана пользователем")
        return gr.update(value="<div class='model-status model-error'>🛑 Загрузка прервана пользователем</div>", visible=True)
    except Exception as e:
        error_msg = f"❌ Ошибка загрузки: {str(e)}"
        print(f"Model loading error: {traceback.format_exc()}")
        return gr.update(value=f"<div class='model-status model-error'>{error_msg}</div>", visible=True)

def on_model_select(model_name):
    """Обработчик выбора модели в dropdown"""
    if not model_name:
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)
    
    return update_model_loading_status(model_name)

def synthesize(
    ref_audio,
    ref_text,
    gen_text,
    remove_silence,
    seed_input_value,
    remember_seed_checkbox,
    model_choice,
    cross_fade_duration=0.15,
    nfe_step=32,
    speed=1.0,
    sway_sampling_coef=-1,
    cfg_strength=2,
    audio_format="wav",
    bitrate="192k",
    progress=gr.Progress()
):
    global stop_generation, last_seed, remember_seed, generation_thread, normalized_audio_cache
    
    remember_seed = remember_seed_checkbox
    
    reset_generation_flags()
    
    try:
        check_stop_generation()
    except KeyboardInterrupt:
        gr.Warning("🛑 Генерация остановлена")
        current_model_status = update_model_loading_status(model_choice)
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
    
    current_model_status = update_model_loading_status(model_choice)
    
    if stop_generation:
        gr.Warning("🛑 Генерация остановлена")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
    
    progress(0.05, desc="🔄 Подготовка...")
    
    if not ref_audio:
        gr.Warning("❌ Пожалуйста, предоставьте референсное аудио")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
    
    if not gen_text or not gen_text.strip():
        gr.Warning("❌ Пожалуйста, введите текст для генерации")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
        
    if not ref_text or not ref_text.strip():
        gr.Warning("❌ Пожалуйста, предоставьте референсный текст")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status

    if gen_text and isinstance(gen_text, str):
        if any(x in gen_text for x in ["/tmp/", ".wav", ".mp3", ".ogg", ".flac"]):
            gr.Warning("⚠️ Поле 'Text to Generate' содержит путь к файлу. Пожалуйста, введите текст вручную.")
            return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status

    if stop_generation:
        gr.Warning("🛑 Генерация остановлена")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status

    current_seed = seed_input_value
    if seed_input_value is None or seed_input_value < 0 or seed_input_value > 2**31 - 1:
        current_seed = np.random.randint(0, 2**31 - 1)
        print(f"🎲 Сгенерирован случайный seed: {current_seed}")
    
    if remember_seed:
        last_seed = current_seed
        print(f"💾 Seed сохранен: {current_seed}")
    
    torch.manual_seed(int(current_seed))
    
    print(f"🎲 Используется seed: {current_seed} (запомнить: {remember_seed})")

    progress(0.08, desc="📝 Подготовка текста...")

    processed_ref_text = ref_text.strip() if ref_text else ""
    processed_gen_text = gen_text.strip() if gen_text else ""
    
    processed_ref_text_final = processed_ref_text
    
    print(f"🎯 Референс текст (как есть): '{processed_ref_text}'")
    print(f"🎯 Генерационный текст (как есть): '{processed_gen_text}'")  
    print("ℹ️ Автоматическая нормализация отключена. Используйте кнопки нормализации перед генерацией.")

    try:
        progress(0.1, desc="🔍 Проверка моделей...")
        
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        progress(0.15, desc="📥 Загрузка модели TTS...")
        model = ensure_model(model_choice, lambda msg, p: progress(p, desc=msg))
        
        if model:
            current_model_status = update_model_loading_status(model_choice)
        
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        progress(0.2, desc="🔊 Загрузка вокодера...")
        vocoder = ensure_vocoder()
        
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
            
    except Exception as e:
        error_msg = f"❌ Ошибка загрузки моделей: {str(e)}"
        gr.Warning(error_msg)
        print(f"Model loading error: {traceback.format_exc()}")
        return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

    if model is None or vocoder is None:
        gr.Warning("❌ Не удалось загрузить модели")
        return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

    device = torch.device("cuda")

    try:
        progress(0.3, desc="🔧 Предобработка аудио...")
        
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        try:
            ref_audio_proc, processed_ref_text_final = preprocess_ref_audio_text(
                ref_audio,
                processed_ref_text,
                show_info=gr.Info
            )
        except Exception as e:
            error_msg = f"❌ Ошибка предобработки: {str(e)}"
            gr.Warning(error_msg)
            traceback.print_exc()
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        progress(0.5, desc="🎵 Генерация аудио...")
        
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        try:
            final_wave, final_sample_rate, combined_spectrogram = infer_process(
                ref_audio_proc,
                processed_ref_text_final,
                processed_gen_text,
                model,
                vocoder,
                cross_fade_duration=cross_fade_duration,
                nfe_step=nfe_step,
                speed=speed,
                sway_sampling_coef=sway_sampling_coef,
                cfg_strength=cfg_strength,
                show_info=gr.Info,
                progress=progress,
            )
        except Exception as e:
            error_msg = f"❌ Ошибка генерации аудио: {str(e)}"
            gr.Warning(error_msg)
            traceback.print_exc()
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        if remove_silence:
            progress(0.7, desc="🔇 Удаление тишины...")
            
            if stop_generation:
                gr.Warning("🛑 Генерация остановлена")
                return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
            
            try:
                with tempfile.NamedTemporaryFile(suffix=".wav", **tempfile_kwargs) as f:
                    temp_path = f.name
                    sf.write(temp_path, final_wave, final_sample_rate)
                    remove_silence_for_generated_wav(temp_path)
                    final_wave_tensor, _ = torchaudio.load(temp_path)
                    final_wave = final_wave_tensor.squeeze().cpu().numpy()
            except Exception as e:
                print("⚠️ Удаление тишины не удалось:", e)

        progress(0.8, desc="💾 Сохранение результата...")
        
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        safe_kwargs = {k: v for k, v in tempfile_kwargs.items() if k != 'delete'}

        with tempfile.NamedTemporaryFile(suffix=f".{audio_format}", delete=False, **safe_kwargs) as tmp:
            temp_output_path = tmp.name

        wave = final_wave
        if wave.ndim == 1:
            channels = 1
        else:
            channels = min(wave.shape[0], 2)
            wave = wave[:channels].T

        if np.max(np.abs(wave)) == 0:
            wave_int16 = np.zeros(wave.shape, dtype=np.int16)
        else:
            wave_int16 = np.int16(wave / np.max(np.abs(wave)) * 32767)

        audio_segment = AudioSegment(
            wave_int16.tobytes(),
            frame_rate=final_sample_rate,
            sample_width=2,
            channels=channels
        )

        try:
            if audio_format == "mp3":
                audio_segment.export(temp_output_path, format="mp3", bitrate=bitrate)
            elif audio_format == "ogg":
                audio_segment.export(temp_output_path, format="ogg", bitrate=bitrate)
            elif audio_format == "flac":
                audio_segment.export(temp_output_path, format="flac")
            else:
                audio_segment.export(temp_output_path, format="wav")
        except Exception as e:
            error_msg = f"❌ Ошибка экспорта аудио: {str(e)}"
            gr.Warning(error_msg)
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        timestamp = int(time.time())
        safe_text = f"audio_{timestamp}"
        final_output_path = os.path.join(os.path.dirname(temp_output_path), f"{safe_text}.{audio_format}")

        try:
            if os.path.exists(final_output_path):
                os.remove(final_output_path)
            os.rename(temp_output_path, final_output_path)
        except Exception as e:
            error_msg = f"❌ Ошибка создания файла: {str(e)}"
            gr.Warning(error_msg)
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        progress(0.9, desc="📊 Сохранение спектрограммы...")

        spectrogram_path = None
        try:
            with tempfile.NamedTemporaryFile(suffix=".png", delete=False, **tempfile_kwargs) as tmp_spectrogram:
                spectrogram_path = tmp_spectrogram.name
            
            save_spectrogram(combined_spectrogram, spectrogram_path)
            print(f"✅ Спектрограмма сохранена: {spectrogram_path}")
            
        except Exception as e:
            print(f"⚠️ Сохранение спектрограммы не удалось: {e}")
            spectrogram_path = None

        progress(1.0, desc="✅ Готово!")

        print("=" * 50)
        print("🎯 ДЕБАГ ИНФОРМАЦИЯ О ФАЙЛАХ:")
        print(f"🎵 Аудио файл: {final_output_path}")
        print(f"🎵 Аудио существует: {os.path.exists(final_output_path)}")

        print(f"📊 Спектрограмма: {spectrogram_path}")
        if spectrogram_path:
            exists = os.path.exists(spectrogram_path)
            print(f"📊 Спектрограмма существует: {exists}")
            if exists:
                size = os.path.getsize(spectrogram_path)
                print(f"📊 Размер спектрограммы: {size} bytes")
        else:
            print("📊 Спектрограмма: None")
        print("=" * 50)

        # 🔴 ОЧИЩАЕМ КЭШ НОРМАЛИЗОВАННЫХ АУДИО ДЛЯ ЭТОГО ФАЙЛА
        if final_output_path in normalized_audio_cache:
            try:
                cached_path = normalized_audio_cache[final_output_path]
                if os.path.exists(cached_path) and cached_path != final_output_path:
                    os.remove(cached_path)
                del normalized_audio_cache[final_output_path]
                print(f"🗑️ Удален кэш нормализованного аудио для: {final_output_path}")
            except:
                pass

        return (
            final_output_path,
            spectrogram_path,
            processed_ref_text_final,
            processed_gen_text,
            gr.update(value=current_seed),
            current_model_status,
            gr.update(visible=True),  # Показываем блок с аудио инструментами
            gr.update(value=""),  # Очищаем статус нормализации
            None  # Нормализованное аудио пока отсутствует
        )

    except Exception as e:
        error_msg = f"❌ Неожиданная ошибка: {str(e)}"
        print(f"Unexpected error in synthesize: {traceback.format_exc()}")
        gr.Warning(error_msg)
        return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status, gr.update(visible=False), gr.update(value=""), None

    finally:
        if not stop_generation:
            try:
                torch.cuda.empty_cache()
                gc.collect()
            except Exception as e:
                print("⚠️ Ошибка при очистке памяти:", e)

def update_model_check_status():
    """Обновить статус проверки моделей"""
    try:
        whisper_ok = test_whisper_model() if asr_processor else False
        
        runorm_ok = False
        try:
            test_text = "13,8 миллиардов лет"
            result = normalizer.norm(test_text)
            runorm_ok = True
        except:
            runorm_ok = False
        
        silero_ok = False
        try:
            test_word = "привет"
            result = accentor(test_word)
            silero_ok = True
        except:
            silero_ok = False
        
        cuda_ok = torch.cuda.is_available()
        
        hf_auth_ok = HF_TOKEN is not None
        
        html = """
        <div style="background: #f8f9fa; padding: 15px; border-radius: 8px;">
            <h3>📊 Статус моделей:</h3>
            <ul style="list-style: none; padding: 0;">
                <li>🎯 <strong>CUDA:</strong> {}</li>
                <li>🔐 <strong>HuggingFace Auth:</strong> {}</li>
                <li>🤖 <strong>Whisper ASR:</strong> {}</li>
                <li>📝 <strong>RUNorm:</strong> {}</li>
                <li>🔊 <strong>Silero Stress:</strong> {}</li>
            </ul>
        </div>
        """.format(
            "✅ Доступен" if cuda_ok else "❌ Недоступен",
            "✅ Авторизован" if hf_auth_ok else "❌ Нет токена",
            "✅ Готов" if whisper_ok else "❌ Не загружен",
            "✅ Готов" if runorm_ok else "⚠️ Частично",
            "✅ Готов" if silero_ok else "⚠️ Частично"
        )
        
        return gr.update(value=html)
    except Exception as e:
        return gr.update(value=f"<div class='model-status model-error'>❌ Ошибка проверки: {str(e)[:100]}</div>")

def update_token_handler(new_token):
    return update_hf_token(new_token)

# ================ ФУНКЦИЯ ДЛЯ ОБНОВЛЕНИЯ ВИДИМОСТИ БИТРЕЙТА ================
def update_bitrate_visibility(audio_format):
    """Обновление видимости выбора битрейта в зависимости от формата"""
    # Битрейт нужен только для mp3 и ogg форматов
    visible = audio_format in ["mp3", "ogg"]
    return gr.update(visible=visible)

# ================ СОЗДАНИЕ ГРАДИО ИНТЕРФЕЙСА ================

with gr.Blocks(title="ESpeech-TTS", css="""
    .error-markdown {
        color: red !important;
        font-weight: bold;
        text-align: center;
        font-size: 18px !important;
        margin: 10px 0;
        animation: fadeIn 0.3s, fadeOut 0.3s 4.7s forwards;
    }
    .model-status {
        padding: 10px;
        border-radius: 5px;
        margin: 10px 0;
        text-align: center;
        font-weight: bold;
    }
    
    .norm-button {
        font-size: 16px !important;
        padding: 12px 24px !important;
        background: linear-gradient(45deg, #007bff, #0056b3) !important;
        color: white !important;
        font-weight: bold !important;
        margin: 10px 0 !important;
        border-radius: 8px !important;
    }
    .norm-button:hover {
        background: linear-gradient(45deg, #0056b3, #003d82) !important;
        transform: translateY(-2px) !important;
        box-shadow: 0 4px 8px rgba(0, 86, 179, 0.3) !important;
    }
    
    .step-result-box {
        background: #f8f9fa !important;
        border-left: 4px solid #28a745 !important;
        border-radius: 5px !important;
        padding: 10px !important;
        margin: 5px 0 !important;
    }
    .step-label {
        font-weight: bold;
        color: #155724;
    }
    .step-text {
        color: #0c5460;
        font-style: italic;
        font-size: 14px;
        word-break: break-word;
    }
    
    .stop-button {
        background: linear-gradient(45deg, #dc3545, #c82333) !important;
        color: white !important;
        font-weight: bold !important;
        border: none !important;
        box-shadow: 0 4px 6px rgba(220, 53, 69, 0.3) !important;
    }
    .stop-button:hover {
        background: linear-gradient(45deg, #c82333, #a71d2a) !important;
        transform: translateY(-2px) !important;
        box-shadow: 0 6px 8px rgba(220, 53, 69, 0.4) !important;
    }
    
    .model-loaded {
        background: #d4edda;
        color: #155724;
        border: 1px solid #c3e6cb;
    }
    .model-error {
        background: #f8d7da;
        color: #721c24;
        border: 1px solid #f5c6cb;
    }
    .model-loading {
        background: #fff3cd;
        color: #856404;
        border: 1px solid #ffeaa7;
    }
    
    .audio-tools-section {
        background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%);
        padding: 20px;
        border-radius: 10px;
        border: 1px solid #dee2e6;
        margin: 20px 0;
        box-shadow: 0 4px 6px rgba(0,0,0,0.05);
    }
               
    /* НОВЫЕ СТИЛИ ДЛЯ АККОРДИОНОВ */
    .normalization-accordion {
        margin-top: 20px !important;  /* ← Добавили отступ сверху */
        border: 1px solid #dee2e6 !important;
        border-radius: 8px !important;
        overflow: hidden !important;
    }

    .normalization-accordion .gr-accordion {
        background: #f8f9fa !important;
    }

    .normalization-accordion .gr-accordion-open {
        background: #e9ecef !important;
    }

    /* Убираем прыжок у всех кнопок при наведении */
    .analyze-button:hover,
    .normalize-button:hover {
        transform: none !important;  /* ← Убираем прыжок */
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2) !important;
    }
    
    .loudness-visualization {
        margin: 15px 0;
        padding: 15px;
        background: white;
        border-radius: 8px;
        border: 1px solid #ced4da;
    }
    
    .normalized-audio-section {
        background: linear-gradient(135deg, #d4edda 0%, #c3e6cb 100%);
        padding: 15px;
        border-radius: 8px;
        border: 2px solid #28a745;
        margin: 15px 0;
    }
    
    .analyze-button {
        background: linear-gradient(45deg, #17a2b8, #138496) !important;
        color: white !important;
        font-weight: bold !important;
        padding: 10px 20px !important;
        border-radius: 6px !important;
    }
    .analyze-button:hover {
        background: linear-gradient(45deg, #138496, #117a8b) !important;
        box-shadow: 0 4px 8px rgba(23, 162, 184, 0.3) !important;
    }
    
    .normalize-button {
        background: linear-gradient(45deg, #ffc107, #e0a800) !important;
        color: white !important;
        font-weight: bold !important;
        padding: 10px 20px !important;
        border-radius: 6px !important;
    }
    .normalize-button:hover {
        background: linear-gradient(45deg, #e0a800, #d39e00) !important;
        box-shadow: 0 4px 8px rgba(255, 193, 7, 0.3) !important;
    }
    
    @keyframes fadeIn { from { opacity: 0; } to { opacity: 1; } }
    @keyframes fadeOut { from { opacity: 1; } to { opacity: 0; } }
    """) as app:
    
    gr.Markdown("# 🎤 ESpeech-TTS - Расширенная версия с инструментами аудио")

    # 🔴 БЛОК СТАТУСА МОДЕЛЕЙ
    with gr.Accordion("📊 Статус моделей", open=False):
        model_check_status = gr.HTML(
            value="<div class='model-status model-loading'>🔄 Проверка моделей...</div>"
        )
        check_models_btn = gr.Button("🔄 Проверить модели", variant="secondary")

    # 🔴 ИНФОРМАЦИЯ О АВТОРИЗАЦИИ
    auth_status_text = "✅ Авторизация HuggingFace выполнена!" if HF_TOKEN else "⚠️ Авторизация HuggingFace не выполнена. Закрытые репозитории недоступны."
    gr.Markdown(f"**🔐 Статус авторизации:** {auth_status_text}")

    gr.Markdown("💡 **Совет:** Добавьте символ '+' для ударения (например, 'прив+ет')")
    gr.Markdown("🎲 **Совет:** Seed отвечает за вариативность генерации. Вы можете подбирать для себя удачное значение или каждый раз использовать случайный seed")
    gr.Markdown("🚀 **CUDA Required:** This application requires GPU with CUDA support")

    # 🔴 БЛОК ДЛЯ ЛОКАЛЬНЫХ МОДЕЛЕЙ
    with gr.Accordion("🔧 Добавить локальную модель", open=False):
        gr.Markdown("### 🔐 Настройка токена HuggingFace")
        gr.Markdown("Токен требуется для доступа к закрытым репозиториям на HuggingFace")
        
        with gr.Row():
            hf_token_input = gr.Textbox(
                label="HuggingFace Token",
                placeholder="hf_xxxxxxxxxxxxxxxxxxxx",
                type="password",
                scale=4
            )
            update_token_btn = gr.Button("🔐 Обновить токен", variant="secondary", scale=1)
            token_status = gr.HTML(visible=False)
        
        gr.Markdown("### 📁 Добавление локальной модели")
        with gr.Row():
            local_model_path = gr.Textbox(
                label="Путь к файлу модели",
                placeholder="hf://SWivid/F5-TTS/F5TTS_v1_Base/model_1250000.safetensors или /path/to/model.safetensors",
                scale=3
            )
            local_vocab_path = gr.Textbox(
                label="Путь к файлу словаря", 
                placeholder="hf://SWivid/F5-TTS/F5TTS_v1_Base/vocab.txt или /path/to/vocab.txt",
                scale=3
            )
            local_model_name = gr.Textbox(
                label="Имя модели",
                placeholder="My Local Model",
                scale=2
            )
        with gr.Row():
            add_local_model_btn = gr.Button("➕ Добавить локальную модель", variant="secondary")
            local_model_status = gr.HTML(visible=False)

    # 🔴 СТАТУС МОДЕЛИ
    all_models = get_all_models_config()
    initial_model = list(all_models.keys())[0] if all_models else "ESpeech-TTS-1_SFT-95K"
    initial_status = update_model_loading_status(initial_model)

    model_status = gr.HTML(
        value=initial_status["value"],
        visible=initial_status["visible"]
    )

    # 🔴 ОСНОВНЫЕ ПОЛЯ ВВОДА
    with gr.Row():
        with gr.Column():
            model_choice = gr.Dropdown(
                choices=list(all_models.keys()),
                value=initial_model,
                label="Выберите модель",
                info="🌐 HuggingFace модели или 🔧 локальные модели"
            )
            
            load_model_btn = gr.Button("🔄 Загрузить выбранную модель", variant="secondary")
            clear_models_btn = gr.Button("🗑️ Очистить загруженные модели", variant="stop")
            
        with gr.Column():
            ref_audio_input = gr.Audio(label="Reference Audio", type="filepath")

            with gr.Row():
                check_audio_btn = gr.Button("🔍 Проверить аудио", variant="secondary")
                transcribe_btn = gr.Button("🎤 Транскрибировать аудио", variant="secondary")
            
            audio_check_status = gr.HTML(visible=False)
            
            ref_text_input = gr.Textbox(
                label="Reference Text",
                lines=2,
                placeholder="Введите текст вручную или используйте кнопку транскрипции"
            )
            
            normalize_ref_btn = gr.Button("📝 Нормализовать референс (RUNorm → Silero)", 
                                        variant="secondary", 
                                        size="lg",
                                        elem_classes="norm-button")

        with gr.Column():
            gen_text_input = gr.Textbox(
                label="Text to Generate",
                lines=5,
                max_lines=20,
                placeholder="Enter text to synthesize..."
            )
    
            normalize_gen_btn = gr.Button("✏️ Нормализовать текст (RUNorm → Silero)", 
                                        variant="secondary", 
                                        size="lg",
                                        elem_classes="norm-button")

    # 🔴 Поля для отображения прогресса нормализации
    with gr.Accordion("📊 Прогресс нормализации референса", open=False):
        ref_norm_step1 = gr.Textbox(label="Шаг 1: RUNorm результат", interactive=False, visible=True)
        ref_norm_step2 = gr.Textbox(label="Шаг 2: Silero Stress результат", interactive=False, visible=True)

    with gr.Accordion("📊 Прогресс нормализации текста", open=False):
        gen_norm_step1 = gr.Textbox(label="Шаг 1: RUNorm результат", interactive=False, visible=True)
        gen_norm_step2 = gr.Textbox(label="Шаг 2: Silero Stress результат", interactive=False, visible=True)

    # 🔴 ДОПОЛНИТЕЛЬНЫЕ НАСТРОЙКИ
    with gr.Accordion("Advanced Settings", open=False):
        with gr.Row():
            seed_input = gr.Number(label="Seed (random)", value=get_current_seed_display(), precision=0)
            remember_seed_checkbox = gr.Checkbox(label="💾 Запомнить этот seed", value=False)
            remove_silence = gr.Checkbox(label="Remove Silences", value=False)
        with gr.Row():
            speed_slider = gr.Slider(label="Speed", minimum=0.3, maximum=2.0, value=1.0, step=0.1)
            nfe_slider = gr.Slider(label="NFE Steps", minimum=4, maximum=64, value=48, step=2)
        with gr.Row():
            cross_fade_slider = gr.Slider(label="Cross-Fade Duration (s)", minimum=0.0, maximum=1.0, value=0.15, step=0.01)
            sway_sampling_slider = gr.Slider(label="Sway Sampling Coef", minimum=-1, maximum=1, value=-1, step=0.1)
        with gr.Row():
            cfg_strength_slider = gr.Slider(label="CFG Strength", minimum=0.5, maximum=5.0, value=2.0, step=0.1)
        with gr.Row():
            audio_format = gr.Radio(["wav", "mp3", "ogg", "flac"], label="Формат", value="wav")
            bitrate = gr.Radio(["128k", "192k", "320k"], label="Битрейт (mp3/ogg)", value="192k", visible=lambda fmt: fmt in ["mp3", "ogg"])

    # 🔴 КНОПКИ ГЕНЕРАЦИИ И ОСТАНОВКИ
    with gr.Row():
        generate_btn = gr.Button("🎤 Generate Speech", variant="primary", size="lg")
        stop_btn = gr.Button("🛑 Stop Generation", variant="stop", size="lg")

    # 🔴 РЕЗУЛЬТАТЫ ГЕНЕРАЦИИ
    with gr.Row():
        audio_output = gr.Audio(label="🎧 Исходное аудио", type="filepath")
        spectrogram_output = gr.Image(label="Spectrogram", type="filepath")

    # 🔴🔴🔴 НОВЫЙ БЛОК: ИНСТРУМЕНТЫ ДЛЯ РАБОТЫ С АУДИО 🔴🔴🔴
    audio_tools_section = gr.Accordion(
        "🛠️ Инструменты для работы с аудио", 
        open=False,  # ← Свернуто по умолчанию
        visible=False,
        elem_classes="audio-tools-section"
    )

    with audio_tools_section:
        # УДАЛИЛИ: gr.Markdown("### 🛠️ Инструменты для работы с аудио")
        # Теперь заголовок в самом Accordion
        
        # 🔴 АНАЛИЗ ГРОМКОСТИ
        with gr.Row():
            analyze_loudness_btn = gr.Button("📊 Анализ громкости", variant="secondary", elem_classes="analyze-button")
        
        loudness_analysis = gr.HTML(label="Результаты анализа громкости", visible=False)
        
        # 🔴 НОРМАЛИЗАЦИЯ АУДИО - отдельный аккордион внутри
        with gr.Accordion("🎚️ Нормализация громкости", open=False, elem_classes="normalization-accordion"):
            with gr.Row():
                with gr.Column(scale=2):
                    target_loudness = gr.Slider(
                        label="Целевая громкость (dBFS)",
                        minimum=-30,
                        maximum=-6,
                        value=-16,
                        step=0.5,
                        info="Рекомендуется: -16 dBFS для подкастов, -14 dBFS для музыки, -18 dBFS для аудиокниг"
                    )
                    headroom_db = gr.Slider(
                        label="Запас до клиппинга (dB)",
                        minimum=0.5,
                        maximum=3.0,
                        value=1.0,
                        step=0.1,
                        info="Запас до максимального уровня для избежания перегрузки"
                    )
                
                with gr.Column(scale=1):
                    normalize_audio_btn = gr.Button("🎚️ Нормализовать запись", variant="secondary", elem_classes="normalize-button")
                    normalize_status = gr.HTML(visible=False)
        
        # 🔴 ПРЕДПРОСМОТР НОРМАЛИЗОВАННОЙ ЗАПИСИ
        with gr.Group(visible=False, elem_classes="normalized-audio-section") as normalized_audio_section:
            gr.Markdown("### ✅ Нормализованная запись готова!")
            normalized_audio_player = gr.Audio(
                label="🎧 Нормализованное аудио (скачать можно через иконку загрузки справа)",
                type="filepath",
                interactive=False
            )

    # ================ ОБРАБОТЧИКИ СОБЫТИЙ ================

    # 1. Обработчик для проверки аудио
    check_audio_btn.click(
        fn=check_audio_file,
        inputs=[ref_audio_input],
        outputs=[audio_check_status]
    )
    
    # 2. Обработчик для проверки моделей
    check_models_btn.click(
        fn=update_model_check_status,
        outputs=[model_check_status]
    )

    # 3. Обработчик для обновления токена
    update_token_btn.click(
        fn=update_token_handler,
        inputs=hf_token_input,
        outputs=token_status
    )

    # 4. Обработчик для добавления локальной модели
    add_local_model_btn.click(
        fn=add_local_model_with_auth,
        inputs=[local_model_path, local_vocab_path, local_model_name, hf_token_input],
        outputs=[model_choice, local_model_status]
    )

    # 5. Обработчик для очистки моделей
    clear_models_btn.click(
        fn=clear_loaded_models,
        outputs=[model_status, local_model_path, local_vocab_path, local_model_name, model_choice]
    )

    # 6. Автозагрузка модели при выборе
    model_choice.change(
        fn=on_model_select,
        inputs=model_choice,
        outputs=model_status
    )

    # 7. Кнопка принудительной загрузки
    load_model_btn.click(
        fn=load_model_with_status,
        inputs=[model_choice],
        outputs=model_status
    )

    # 8. Кнопка транскрипции
    transcribe_btn.click(
        fn=manual_transcribe_audio,
        inputs=[ref_audio_input],
        outputs=[ref_text_input, audio_check_status]
    )

    # 9. Кнопка нормализации референса
    normalize_ref_btn.click(
        fn=normalize_ref_text_with_steps,
        inputs=[ref_text_input],
        outputs=[ref_text_input, ref_norm_step1, ref_norm_step2]
    )

    # 10. Кнопка нормализации текста генерации
    normalize_gen_btn.click(
        fn=lambda text: (
            print("\n" + "="*60),
            print("🔴 [COLAB DEBUG] КНОПКА НОРМАЛИЗАЦИИ НАЖАТА"),
            print(f"📖 Полученный текст: '{text}'"),
            print(f"📊 Длина текста: {len(text) if text else 0}"),
            print("="*60),
            
            print("🔍 Проверка моделей..."),
            print(f"  • RUNorm доступен: {hasattr(normalizer, 'norm')}"),
            print(f"  • Silero доступен: {accentor is not None}"),
            
            (print("🔍 Анализ текста:"),
            print(f"  • Содержит цифры: {any(char.isdigit() for char in text)}"),
            print(f"  • Содержит ударения (+): {'+' in text}"),
            print(f"  • Содержит скобки []: {'[' in text or ']' in text}")) if text else None,
            
            result := normalize_gen_text_with_steps(text),
            
            print("\n" + "="*60),
            print("✅ [DEBUG COLAB] НОРМАЛИЗАЦИЯ ЗАВЕРШЕНА"),
            print(f"📤 Результат возвращается в интерфейс"),
            print(f"📝 Шаг 1: {result[1][:50]}..."),
            print(f"📝 Шаг 2: {result[2][:50]}..."),
            print("="*60),
            
            result
        )[-1],
        inputs=[gen_text_input],
        outputs=[gen_text_input, gen_norm_step1, gen_norm_step2]
    )

    # 11. Обработчик изменения формата
    audio_format.change(
        update_bitrate_visibility,
        inputs=audio_format,
        outputs=bitrate
    )

    # 12. Обработчик остановки
    stop_btn.click(
        fn=stop_generation_process,
        outputs=None,
        queue=False
    )

    # 13. 🔴 НОВЫЙ ОБРАБОТЧИК: Анализ громкости аудио
    def analyze_audio_handler(audio_path):
        """Обработчик для анализа громкости аудио"""
        if not audio_path:
            return (
                gr.update(visible=False),
                gr.update(value="❌ Нет аудиофайла для анализа", visible=True)
            )
        
        try:
            loudness, max_loudness, dynamic_range, analysis_report, visualization = analyze_audio_loudness(audio_path)
            
            if analysis_report:
                return (
                    gr.update(value=analysis_report, visible=True),
                    gr.update(visible=True)
                )
            else:
                return (
                    gr.update(visible=False),
                    gr.update(value="❌ Не удалось проанализировать аудио", visible=True)
                )
                
        except Exception as e:
            print(f"Ошибка анализа громкости: {e}")
            return (
                gr.update(visible=False),
                gr.update(value=f"❌ Ошибка анализа: {str(e)[:100]}", visible=True)
            )

    analyze_loudness_btn.click(
        fn=analyze_audio_handler,
        inputs=[audio_output],
        outputs=[loudness_analysis, normalize_status]
    )

    # 14. 🔴 НОВЫЙ ОБРАБОТЧИК: Нормализация аудио
    def normalize_audio_handler(audio_path, target_loudness, headroom_db_value):
        """Обработчик для нормализации аудио"""
        if not audio_path:
            return (
                gr.update(visible=False),
                gr.update(value="❌ Нет аудиофайла для нормализации", visible=True),
                None,
                gr.update(visible=False)
            )
        
        try:
            # Определяем формат из исходного файла
            audio_format_from_file = os.path.splitext(audio_path)[1].lower().replace('.', '')
            if audio_format_from_file not in ['wav', 'mp3', 'ogg', 'flac']:
                audio_format_from_file = 'wav'
            
            print(f"🎚️ Начинаем нормализацию аудио. Формат: {audio_format_from_file}")
            print(f"🎯 Целевая громкость: {target_loudness} dBFS")
            print(f"🛡️ Запас до клиппинга: {headroom_db_value} dB")
            
            normalized_path, report = normalize_audio_enhanced(
                audio_path, 
                target_loudness=target_loudness,
                headroom=headroom_db_value,
                audio_format=audio_format_from_file
            )
            
            if normalized_path and os.path.exists(normalized_path):
                return (
                    gr.update(value=report, visible=True),
                    gr.update(value="✅ Нормализация завершена!", visible=True),
                    normalized_path,
                    gr.update(visible=True)
                )
            else:
                return (
                    gr.update(visible=False),
                    gr.update(value=f"❌ Не удалось нормализовать аудио: {report}", visible=True),
                    None,
                    gr.update(visible=False)
                )
                
        except Exception as e:
            error_msg = f"❌ Ошибка нормализации: {str(e)[:100]}"
            print(f"Ошибка нормализации: {traceback.format_exc()}")
            return (
                gr.update(visible=False),
                gr.update(value=error_msg, visible=True),
                None,
                gr.update(visible=False)
            )

    normalize_audio_btn.click(
        fn=normalize_audio_handler,
        inputs=[audio_output, target_loudness, headroom_db],
        outputs=[loudness_analysis, normalize_status, normalized_audio_player, normalized_audio_section]
    )

    # 16. 🔴 ИСПРАВЛЕННЫЙ ОБРАБОТЧИК ГЕНЕРАЦИИ
    generate_btn.click(
        synthesize,
        inputs=[
            ref_audio_input,
            ref_text_input,
            gen_text_input,
            remove_silence,
            seed_input,
            remember_seed_checkbox,
            model_choice,
            cross_fade_slider,
            nfe_slider,
            speed_slider,
            sway_sampling_slider,
            cfg_strength_slider,
            audio_format,
            bitrate,
        ],
        outputs=[
            audio_output, 
            spectrogram_output, 
            ref_text_input, 
            gen_text_input, 
            seed_input,
            model_status,
            audio_tools_section,  # ← ВОЗВРАЩАЕМ САМ КОМПОНЕНТ, не gr.update()
            normalize_status,
            normalized_audio_player
        ]
    ).then(
        # После генерации ОТКРЫВАЕМ аккордион
        fn=lambda: gr.update(open=True),
        inputs=[],
        outputs=[audio_tools_section],
        queue=False
    ).then(
        # Скрываем результаты предыдущей нормализации
        fn=lambda: (
            gr.update(visible=False),
            gr.update(visible=False),
        ),
        outputs=[loudness_analysis, normalized_audio_section]
    )

if __name__ == "__main__":
    print("\n" + "="*60)
    print("🚀 ПОДГОТОВКА К ЗАПУСКУ")
    print("="*60)
    
    models_ok = check_all_models_before_launch()
    
    if models_ok:
        print("\n✅ Все модели проверены, запускаем интерфейс...")
        app.launch(share=True)
    else:
        print("\n⚠️ Некоторые модели не работают корректно")
        print("   Интерфейс будет запущен с ограниченной функциональностью")
        app.launch(share=True)

Checking CUDA availability...
CUDA is available. Using device: Tesla T4
Preloading model...
CUDA is available. Using device: Tesla T4
Trying to download model file 'espeech_tts_rlv2.pt' and 'vocab.txt' from hub 'ESpeech/ESpeech-TTS-1_RL-V2'


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Downloaded model to /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/espeech_tts_rlv2.pt
Downloaded vocab to /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/vocab.txt
Loading model from: /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/espeech_tts_rlv2.pt

vocab :  /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/vocab.txt
token :  custom
model :  /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/espeech_tts_rlv2.pt 

Model loaded and moved to CUDA: cuda
Model preloaded.
Loading RUAccent...
RUAccent loaded.
Preloading vocoder...
CUDA is available. Using device: Tesla T4
Loading vocoder...
Download Vocos from huggingface charactr/vocos-mel-24khz
Vocoder